<a href="https://colab.research.google.com/github/ekat92/Book-Project-Econometrics-and-ML/blob/alpha_rnns/alpha_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Alpha_RNNs-Bitcoin
# Author: Chris Zhu
# Version: 1.1 (17.12.2020)
# License: MIT
# Email: yzhu22@colby.edu
# Notes: tested on Mac OS X with Python 3.6 and Tensorflow 2.2 Keras 2.3.1

## Overview
This notebook provides a toy example for $\alpha$-RNNs network to predict

In [ ]:
#Only needed if the notebook is being run on Google Golab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import sys
path = '/content/drive/My Drive/Book-Project-Econometrics-and-ML' # set the path to location of the data file coinbase.csv
sys.path.append(path)

In [4]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import statsmodels.api as sm
import tensorflow as tf
import statsmodels.api as sm
import os

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import Layer, LSTM, GRU, SimpleRNN
from keras import optimizers
from keras.regularizers import l1,l2
from keras import regularizers
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.layers import Layer,RNN
from keras import backend as K
from keras import layers
import keras.layers
import tensorflow.keras as keras 
from keras.legacy import interfaces
from keras import *
# Custom classes for alpha-RNNs and alpha_t-RNNs
# Custom classes for alpha-RNNs and alpha_t-RNNs
from alphaRNN import *
from alphatRNN import *

In [ ]:
%matplotlib inline